# Imports

In [13]:
import cv2
import torch
import ultralytics
from ultralytics import YOLO


In [7]:
torch.cuda.is_available()


False

In [8]:
ultralytics.checks()

Ultralytics YOLOv8.0.124  Python-3.11.2 torch-2.0.1+cpu CPU
Setup complete  (8 CPUs, 15.8 GB RAM, 248.9/475.7 GB disk)


# Setting up the model

In [14]:
# Load a model
model = YOLO("yolov8n-pose.pt")  # This model is pretrained. To setup a empty model use : model = YOLO('yolov8n.yaml')
model.info()


YOLOv8n-pose summary: 250 layers, 3295470 parameters, 0 gradients


(250, 3295470, 0, 0.0)

# Training

The COCO8-Pose dataset is a tiny dataset ment for testing pipelines for errors or as a check before training larger datasets.
It is a good dataset to show how easy it is to train a dataset using ultralytics.

In [10]:
# Train the model
model.train(data='coco8-pose.yaml', epochs=100, imgsz=640, plots=True)

Ultralytics YOLOv8.0.124  Python-3.11.2 torch-2.0.1+cpu CPU
yolo\engine\trainer: task=pose, mode=train, model=yolov8n-pose.pt, data=coco8-pose.yaml, epochs=100, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=False, optimize=False, int8=False, dynamic=False, simplify=False, opset=None,

# Validate
The re-trained YOLOv8n-pose model accuracy on the COCO8-pose dataset.

In [15]:
model = YOLO('runs/pose/train/weights/best.pt')  # load a custom model

# Validate the model
metrics = model.val()  # no arguments needed, dataset and settings remembered



Ultralytics YOLOv8.0.124  Python-3.11.2 torch-2.0.1+cpu CPU
YOLOv8n-pose summary (fused): 187 layers, 3289964 parameters, 0 gradients
val: Scanning C:\Users\dherm\python\AI2_exam_assignment\datasets\coco8-pose\labels\val.cache... 4 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4/4 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Pose(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  1.09it/s]
                   all          4         14      0.927      0.906      0.907      0.668      0.845        0.5      0.535      0.352
Speed: 2.8ms preprocess, 207.7ms inference, 0.0ms loss, 1.6ms postprocess per image
Results saved to runs\pose\val2


# Predict

In [24]:
# Predict with the model
predict = model.predict('https://ultralytics.com/images/bus.jpg', conf=0.75)
# predict on an image
print(predict)


Found https:\ultralytics.com\images\bus.jpg locally at bus.jpg
image 1/1 C:\Users\dherm\python\AI2_exam_assignment\bus.jpg: 640x480 3 persons, 156.3ms
Speed: 2.0ms preprocess, 156.3ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 480)


[ultralytics.yolo.engine.results.Results object with attributes:

boxes: ultralytics.yolo.engine.results.Boxes object
keypoints: ultralytics.yolo.engine.results.Keypoints object
keys: ['boxes', 'keypoints']
masks: None
names: {0: 'person'}
orig_img: array([[[122, 148, 172],
        [120, 146, 170],
        [125, 153, 177],
        ...,
        [157, 170, 184],
        [158, 171, 185],
        [158, 171, 185]],

       [[127, 153, 177],
        [124, 150, 174],
        [127, 155, 179],
        ...,
        [158, 171, 185],
        [159, 172, 186],
        [159, 172, 186]],

       [[128, 154, 178],
        [126, 152, 176],
        [126, 154, 178],
        ...,
        [158, 171, 185],
        [158, 171, 185],
        [158, 171, 185]],

       ...,

       [[185, 185, 191],
        [182, 182, 188],
        [179, 179, 185],
        ...,
        [114, 107, 112],
        [115, 105, 111],
        [116, 106, 112]],

       [[157, 157, 163],
        [180, 180, 186],
        [185, 186, 190],
  

In [25]:
# Show the image with keypoint names

img = cv2.imread('bus.jpg')

results = model('bus.jpg', conf=0.75)[0]

for result in results:
    for keypoint_index, keypoint in enumerate(result.keypoints.data[0].tolist()):
        cv2.putText(img, str(keypoint_index), (int(keypoint[0]), int(keypoint[1])),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

cv2.imshow('Results', img)
cv2.waitKey(0)
cv2.destroyAllWindows()


image 1/1 C:\Users\dherm\python\AI2_exam_assignment\bus.jpg: 640x480 3 persons, 188.6ms
Speed: 2.0ms preprocess, 188.6ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 480)
